Installing Dependencies

In [1]:
import gym 
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete 
import numpy as np
import random
import os
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy

Test DATA

In [8]:
import pandas as pd

# Sample data
data = {
    'speed of operation': [0, 1, 2, 3, 4],
    'load condition': [0,1,0,0,1],
    'Speed condition': [1,0,1,0,1],
    'batch process': [1, 1, 1, 0, 0],
    'mounting condition': [0, 1, 2, 1, 0],
    'Infinite Uptime pred': [0,0,1,1,0],
    'customer feedback':[0,1,0,1,0]
}

# Create the DataFrame
df = pd.DataFrame(data)

In [9]:
df

,speed of operation,load condition,Speed condition,batch process,mounting condition,Infinite Uptime pred,customer feedback
0,0,0,1,1,0,0,0
1,1,1,0,1,1,0,1
2,2,0,1,1,2,1,0
3,3,0,0,0,1,1,1
4,4,1,1,0,0,0,0


In [12]:
df.iloc[0].tolist()

[0, 0, 1, 1, 0, 0, 0]

In [11]:
MultiDiscrete([5,2,2,2,3,2,2]).sample()

array([3, 0, 0, 1, 0, 1, 0], dtype=int64)

Building Environment

In [28]:
class TestEnv(Env):
    def __init__(self):
        # Actions we can alert(1),no alert(0)
        self.action_space = Discrete(2)
        # feature array
        self.observation_space = MultiDiscrete([5,2,2,2,3,2,2])
        # Set start state
        self.state = df.iloc[0].tolist()
        # Set run length
        self.length = 5

        
    def step(self, action):
        self.state=df.iloc[5-self.length].tolist()
        self.length -= 1 
        
        # Calculate reward
        if self.state[6] ==action :
            reward =1 
        else: 
            reward = -1 
        

        if self.length <= 0: 
            done = True
        else:
            done = False
        
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        self.state = np.array(df.iloc[0].tolist()).astype(int)
        self.length = 5 
        return self.state

In [29]:
env=TestEnv()

In [20]:
env.observation_space.sample()

array([3, 0, 0, 0, 2, 1, 1], dtype=int64)

In [25]:
env.action_space.sample()


1

In [26]:
env.reset()

array([0, 0, 1, 1, 0, 0, 0])

Testing Environment

In [33]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:3
Episode:2 Score:-1
Episode:3 Score:3
Episode:4 Score:-1
Episode:5 Score:-3


Training RL model

In [34]:
log_path = os.path.join('Training', 'Logs')
model = PPO("MlpPolicy", env, verbose=1, tensorboard_log=log_path)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


C:\Users\advan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [35]:
model.learn(total_timesteps=40000)

Logging to Training\Logs\PPO_1
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 5        |
|    ep_rew_mean     | -0.04    |
| time/              |          |
|    fps             | 814      |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 5           |
|    ep_rew_mean          | 0.96        |
| time/                   |             |
|    fps                  | 596         |
|    iterations           | 2           |
|    time_elapsed         | 6           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.021682106 |
|    clip_fraction        | 0.521       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.671      |
|    explained_variance   | -0.00839    |

In [36]:
model.save(os.path.join('Training','Saved Models','V1'))